In [1]:
import os
import io
import random
import json
import base64
import subprocess
from pickle import dump
from shutil import rmtree, copy2

import cv2
import boto3
import mxnet as mx
import numpy as np
from pandas import read_csv
import sagemaker

In [2]:
sagemaker_session = sagemaker.Session()

## Synthetic Data Generation

In [3]:
dataset_size = 12000
img_size = 128
shape_size = 8
max_rows = 14
max_cols = 14
target_shape = "circle"

In [4]:
shapes = ["circle", "triangle", "square"]
img_width = img_height = img_size

In [5]:
%%time

os.makedirs("data/raw")
    
random.seed(42)

with open("data/dataset.csv", "w+") as dataset:
    dataset.write("img_path,target\n")
    for i in range(1, dataset_size + 1):
        target_number_of_shapes = 0

        filename = str(i).zfill(6) + ".shape"

        file_content = "img_dim:{},shp_dim:{}>>>".format(img_size, shape_size)
        for x in range(0, random.randint(1, max_rows)):
            for y in range(0, random.randint(1, max_cols)):
                shape_idx = random.randint(0, len(shapes) - 1)
                shape = shapes[shape_idx]
                if shape == target_shape:
                    target_number_of_shapes += 1
                file_content = file_content + shape + ","
            file_content = file_content[:-1]
            file_content += "|"
        file_content = file_content[:-1]
        file_content += "<<<"

        with open("./data/raw/{}".format(filename), "w") as shape_file:
            shape_file.write(file_content)

        dataset.write("./raw/{}.png,{}\n".format(filename, str(target_number_of_shapes)))

CPU times: user 2.89 s, sys: 2.12 s, total: 5.01 s
Wall time: 5.13 s


In [6]:
%%time
subprocess.call("""
    java -cp bin/shaper-all.jar com.cosminsanda.shaper.compiler.Shaper2Image --source-dir data/raw""", shell=True);

CPU times: user 3.12 ms, sys: 10.5 ms, total: 13.6 ms
Wall time: 49.8 s


0

In [7]:
df = read_csv("data/dataset.csv")

train = df.sample(frac=.8333, random_state=42)
validation = df.loc[~df.index.isin(train.index), :].sample(frac=.5, random_state=42)
test = df.loc[np.logical_not(np.logical_xor(~df.index.isin(train.index), ~df.index.isin(validation.index))), :]

In [8]:
def transform(row):
    img = cv2.imread("./data/{}".format(row["img_path"]))
    img = mx.nd.array(img)
    img = img.astype(np.float32)
    img = mx.nd.transpose(img, (2, 0, 1))
    img = img / 255
    label = np.float32(row["target"])
    return img, label

In [9]:
%%time
train_nd = [transform(row) for _, row in train.iterrows()]
validation_nd = [transform(row) for _, row in validation.iterrows()]

CPU times: user 10.3 s, sys: 2.66 s, total: 13 s
Wall time: 11.7 s


In [10]:
def save_to_disk(data, type):
    os.makedirs("data/pickles/{}".format(type))
    with open("data/pickles/{}/data.p".format(type), "wb") as out:
        dump(data, out)

In [11]:
%%time
save_to_disk(train_nd, "train")
save_to_disk(validation_nd, "validation")

CPU times: user 3.74 s, sys: 7.38 s, total: 11.1 s
Wall time: 12.4 s


In [12]:
%%time
inputs = sagemaker_session.upload_data(path="data/pickles", bucket="redacted", key_prefix="sagemaker/demo")

CPU times: user 23.9 s, sys: 19.1 s, total: 43 s
Wall time: 5min 17s


In [13]:
rmtree("./test", True)
os.makedirs("./test")
for _, row in test.iterrows():
    os.makedirs("test/{}".format(row["target"]), exist_ok=True)
    copy2("./data/{}".format(row["img_path"]), "./test/{}".format(row["target"]))

In [14]:
rmtree("data", True)

## Testing Inference

In [25]:
sagemaker_runtime_client = boto3.client("sagemaker-runtime")

In [27]:
for directory in os.listdir("./test"):
    batch = []
    for file in os.listdir("./test/{}".format(directory)):
        with open("./test/{}/{}".format(directory, file), "rb") as image_file:
            batch.append(base64.b64encode(image_file.read()).decode("utf-8"))
    binary_json = json.dumps(batch).encode("utf-8")
    response = sagemaker_runtime_client.invoke_endpoint(
        EndpointName=predictor.endpoint,
        Body=binary_json,
        ContentType="application/json",
        Accept="application/json"
    )["Body"].read()
    individual_predictions = json.loads(response, encoding="utf-8")
    total = 0
    detected = 0
    for prediction in individual_predictions:
        total += 1 
        if int(prediction) == int(directory):
            detected += 1
    print("""Images with {} circles:
        Total: {}
        Detected: {}
        Accuracy: {:0.2f}
    """.format(directory, str(total), str(detected), detected/total))

Images with 50 circles:
        Total: 2
        Detected: 0
        Accuracy: 0.00
    
Images with 32 circles:
        Total: 23
        Detected: 13
        Accuracy: 0.57
    
Images with 35 circles:
        Total: 12
        Detected: 8
        Accuracy: 0.67
    
Images with 34 circles:
        Total: 15
        Detected: 12
        Accuracy: 0.80
    
Images with 33 circles:
        Total: 17
        Detected: 12
        Accuracy: 0.71
    
Images with 20 circles:
        Total: 36
        Detected: 31
        Accuracy: 0.86
    
Images with 18 circles:
        Total: 25
        Detected: 22
        Accuracy: 0.88
    
Images with 27 circles:
        Total: 25
        Detected: 20
        Accuracy: 0.80
    
Images with 9 circles:
        Total: 30
        Detected: 29
        Accuracy: 0.97
    
Images with 0 circles:
        Total: 9
        Detected: 9
        Accuracy: 1.00
    
Images with 11 circles:
        Total: 30
        Detected: 30
        Accuracy: 1.00
    
Images